In [ ]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_izq = [27, 28, 29, 30, 31]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_izq = calcular_centro_region(landmarks, oreja_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja izquierda
        ear_left_points = np.array([landmarks[i] for i in oreja_izq], dtype=np.float32)
        ear_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_left_points), 1))
        points_ones = np.hstack([ear_left_points, ones])
        ear_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja izquierda
        centro_x, centro_y = calcular_centro_region(ear_left_points_rotated, range(len(oreja_izq)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja izquierda al nuevo recorte
        ear_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_left_points_scaled)
            image = augmented['image']
            ear_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EarLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarLeft().cuda()
optimizer = optim.Adamax(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja izquierda)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_left_points, _, _, _ in train_dataloader:
            images, ear_left_points = images.cuda(), ear_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_left_points, _, _, _ in val_dataloader:
                    val_images, val_ear_left_points = val_images.cuda(), val_ear_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_left_points.cpu().numpy().reshape(-1, len(oreja_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarLeftLandmarks V3.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

Epoch [1/300], Loss: 18580.1025
Validation Loss: 18264.7334
NME: 1.5296
Best model saved at epoch 1 with val_loss 18264.7334
Epoch [2/300], Loss: 17232.5448
Validation Loss: 15836.9244
NME: 1.4316
Best model saved at epoch 2 with val_loss 15836.9244
Epoch [3/300], Loss: 13689.1608
Validation Loss: 10910.0747
NME: 1.1746
Best model saved at epoch 3 with val_loss 10910.0747
Epoch [4/300], Loss: 8154.5957
Validation Loss: 5186.2174
NME: 0.7490
Best model saved at epoch 4 with val_loss 5186.2174
Epoch [5/300], Loss: 3727.8263
Validation Loss: 2360.8130
NME: 0.4283
Best model saved at epoch 5 with val_loss 2360.8130
Epoch [6/300], Loss: 2174.2569
Validation Loss: 1699.5994
NME: 0.3509
Best model saved at epoch 6 with val_loss 1699.5994
Epoch [7/300], Loss: 1760.5794
Validation Loss: 1491.0681
NME: 0.3393
Best model saved at epoch 7 with val_loss 1491.0681
Epoch [8/300], Loss: 1629.6486
Validation Loss: 1407.7641
NME: 0.3398
Best model saved at epoch 8 with val_loss 1407.7641
Epoch [9/300], 

In [3]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
oreja_der = [22, 23, 24, 25, 26]  # Cambiado a oreja derecha

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EarRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_oreja_der = calcular_centro_region(landmarks, oreja_der)  # Cambiado a oreja derecha
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_oreja_der -= np.array([x_min, y_min])  # Cambiado a oreja derecha
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la oreja derecha
        ear_right_points = np.array([landmarks[i] for i in oreja_der], dtype=np.float32)  # Cambiado a oreja derecha
        ear_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(ear_right_points), 1))
        points_ones = np.hstack([ear_right_points, ones])
        ear_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la oreja derecha
        centro_x, centro_y = calcular_centro_region(ear_right_points_rotated, range(len(oreja_der)))  # Cambiado a oreja derecha

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_ear = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_ear, (224, 224))

        # Ajustar los puntos faciales de la oreja derecha al nuevo recorte
        ear_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in ear_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        ear_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in ear_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=ear_right_points_scaled)
            image = augmented['image']
            ear_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(ear_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EarRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EarRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EarRight(nn.Module):
    def __init__(self):
        super(DenseNet121EarRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(oreja_der) * 2)  # Cambiado a oreja derecha
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EarRight().cuda()
optimizer = optim.Adamax(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para oreja derecha)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, ear_right_points, _, _, _ in train_dataloader:  # Cambiado a oreja derecha
            images, ear_right_points = images.cuda(), ear_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, ear_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_ear_right_points, _, _, _ in val_dataloader:  # Cambiado a oreja derecha
                    val_images, val_ear_right_points = val_images.cuda(), val_ear_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_ear_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_ear_right_points.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(oreja_der), 2))  # Cambiado a oreja derecha

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(oreja_der))  # Cambiado a oreja derecha
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EarRightLandmarks V3.pth')  # Cambiado a oreja derecha
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 16192.0011
Validation Loss: 15823.5184
NME: 1.2604
Best model saved at epoch 1 with val_loss 15823.5184
Epoch [2/300], Loss: 14375.6213
Validation Loss: 13012.2011
NME: 1.1824
Best model saved at epoch 2 with val_loss 13012.2011
Epoch [3/300], Loss: 10416.5522
Validation Loss: 8376.8387
NME: 0.9797
Best model saved at epoch 3 with val_loss 8376.8387
Epoch [4/300], Loss: 5671.8928
Validation Loss: 3423.1829
NME: 0.5915
Best model saved at epoch 4 with val_loss 3423.1829
Epoch [5/300], Loss: 2348.4614
Validation Loss: 1499.0376
NME: 0.2963
Best model saved at epoch 5 with val_loss 1499.0376
Epoch [6/300], Loss: 1502.2516
Validation Loss: 1237.2129
NME: 0.2688
Best model saved at epoch 6 with val_loss 1237.2129
Epoch [7/300], Loss: 1337.1945
Validation Loss: 1152.7776
NME: 0.2622
Best model saved at epoch 7 with val_loss 1152.7776
Epoch [8/300], Loss: 1317.7789
Validation Loss: 1124.7537
NME: 0.2647
Best model saved at epoch 8 with val_loss 1124.7537
Epoch [9/300], Lo

In [1]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeLeftDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo izquierdo
        eye_left_points = np.array([landmarks[i] for i in ojo_izq], dtype=np.float32)
        eye_left_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_left_points), 1))
        points_ones = np.hstack([eye_left_points, ones])
        eye_left_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo izquierdo
        centro_x, centro_y = calcular_centro_region(eye_left_points_rotated, range(len(ojo_izq)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo izquierdo al nuevo recorte
        eye_left_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_left_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_left_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_left_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_left_points_scaled)
            image = augmented['image']
            eye_left_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_left_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeLeftDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeLeftDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeLeft(nn.Module):
    def __init__(self):
        super(DenseNet121EyeLeft, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_izq) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeLeft().cuda()
optimizer = optim.Adamax(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo izquierdo)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_left_points, _, _, _ in train_dataloader:
            images, eye_left_points = images.cuda(), eye_left_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_left_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_left_points, _, _, _ in val_dataloader:
                    val_images, val_eye_left_points = val_images.cuda(), val_eye_left_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_left_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_left_points.cpu().numpy().reshape(-1, len(ojo_izq), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_izq), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_izq))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeLeftLandmarks V3.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` 

Epoch [1/300], Loss: 14645.3623
Validation Loss: 14082.6291
NME: 7.3234
Best model saved at epoch 1 with val_loss 14082.6291
Epoch [2/300], Loss: 12694.1615
Validation Loss: 10678.9164
NME: 6.4475
Best model saved at epoch 2 with val_loss 10678.9164
Epoch [3/300], Loss: 7792.6973
Validation Loss: 4449.0203
NME: 3.8791
Best model saved at epoch 3 with val_loss 4449.0203
Epoch [4/300], Loss: 2352.0898
Validation Loss: 1133.5370
NME: 1.4848
Best model saved at epoch 4 with val_loss 1133.5370
Epoch [5/300], Loss: 881.2938
Validation Loss: 681.7194
NME: 0.9981
Best model saved at epoch 5 with val_loss 681.7194
Epoch [6/300], Loss: 669.4985
Validation Loss: 586.9979
NME: 0.9569
Best model saved at epoch 6 with val_loss 586.9979
Epoch [7/300], Loss: 626.2771
Validation Loss: 565.0573
NME: 0.9966
Best model saved at epoch 7 with val_loss 565.0573
Epoch [8/300], Loss: 615.2845
Validation Loss: 561.0078
NME: 1.0191
Best model saved at epoch 8 with val_loss 561.0078
Epoch [9/300], Loss: 613.3866


In [2]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class EyeRightDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave del ojo derecho
        eye_right_points = np.array([landmarks[i] for i in ojo_der], dtype=np.float32)
        eye_right_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(eye_right_points), 1))
        points_ones = np.hstack([eye_right_points, ones])
        eye_right_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado del ojo derecho
        centro_x, centro_y = calcular_centro_region(eye_right_points_rotated, range(len(ojo_der)))

        # Definir los límites del recorte de 56x56 píxeles
        half_crop_size = 28
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_eye = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_eye, (224, 224))

        # Ajustar los puntos faciales del ojo derecho al nuevo recorte
        eye_right_points_adjusted = [(p[0] - x1, p[1] - y1) for p in eye_right_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        eye_right_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in eye_right_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=eye_right_points_scaled)
            image = augmented['image']
            eye_right_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(eye_right_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = EyeRightDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = EyeRightDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121EyeRight(nn.Module):
    def __init__(self):
        super(DenseNet121EyeRight, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(ojo_der) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121EyeRight().cuda()
optimizer = optim.Adamax(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para ojo derecho)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, eye_right_points, _, _, _ in train_dataloader:
            images, eye_right_points = images.cuda(), eye_right_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, eye_right_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_eye_right_points, _, _, _ in val_dataloader:
                    val_images, val_eye_right_points = val_images.cuda(), val_eye_right_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_eye_right_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_eye_right_points.cpu().numpy().reshape(-1, len(ojo_der), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(ojo_der), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(ojo_der))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'EyeRightLandmarks V3.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 14734.0587
Validation Loss: 14261.8735
NME: 7.3281
Best model saved at epoch 1 with val_loss 14261.8735
Epoch [2/300], Loss: 13062.6246
Validation Loss: 11191.3888
NME: 6.4099
Best model saved at epoch 2 with val_loss 11191.3888
Epoch [3/300], Loss: 8761.4587
Validation Loss: 5664.5527
NME: 4.2109
Best model saved at epoch 3 with val_loss 5664.5527
Epoch [4/300], Loss: 3565.9724
Validation Loss: 2106.3515
NME: 2.0268
Best model saved at epoch 4 with val_loss 2106.3515
Epoch [5/300], Loss: 1603.0103
Validation Loss: 1178.1881
NME: 1.3132
Best model saved at epoch 5 with val_loss 1178.1881
Epoch [6/300], Loss: 1011.1178
Validation Loss: 800.8623
NME: 1.0178
Best model saved at epoch 6 with val_loss 800.8623
Epoch [7/300], Loss: 761.8532
Validation Loss: 638.9376
NME: 0.9486
Best model saved at epoch 7 with val_loss 638.9376
Epoch [8/300], Loss: 678.5910
Validation Loss: 588.5069
NME: 0.9748
Best model saved at epoch 8 with val_loss 588.5069
Epoch [9/300], Loss: 648.2

In [3]:
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim
from torchvision import models
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# Rutas
train_img_dir = r'C:\CatFLW dataset\train\images'
train_labels_dir = r'C:\CatFLW dataset\train\labels'
val_img_dir = r'C:\CatFLW dataset\val\images'
val_labels_dir = r'C:\CatFLW dataset\val\labels'

# Definir los índices de las regiones según los puntos proporcionados
ojo_der = [3, 4, 5, 6, 7, 36, 37, 38]
ojo_izq = [1, 8, 9, 10, 11, 39, 40, 41]
nariz = [0, 2, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 32, 33, 34, 35, 42, 43, 44, 45, 46, 47]

# Función para calcular el punto promedio de una región
def calcular_centro_region(landmarks, indices):
    puntos = [landmarks[i] for i in indices]
    centro = np.mean(puntos, axis=0)
    return centro

# Dataset personalizado
class NoseDataset(Dataset):
    def __init__(self, img_dir, labels_dir, transform=None):
        self.img_dir = img_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(img_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.labels_dir, img_name.replace('.png', '.json'))
        
        # Cargar imagen
        image = np.array(Image.open(img_path).convert('RGB'))
        
        # Cargar etiquetas y puntos de referencia
        with open(label_path, 'r') as f:
            labels = json.load(f)
            landmarks = np.array(labels['labels'])  # Convertir a array de NumPy
            bounding_box = labels['bounding_boxes']
        
        # Recortar la imagen usando el bounding box
        x_min, y_min, x_max, y_max = map(int, bounding_box)
        image_cropped = image[y_min:y_max, x_min:x_max]
        
        # Calcular dimensiones originales
        original_height, original_width = image_cropped.shape[:2]
        
        # Calcular centros de los ojos y la nariz
        centro_ojo_der = calcular_centro_region(landmarks, ojo_der)
        centro_ojo_izq = calcular_centro_region(landmarks, ojo_izq)
        centro_nariz = calcular_centro_region(landmarks, nariz)
        
        # Ajustar coordenadas de los centros al recorte
        centro_ojo_der -= np.array([x_min, y_min])
        centro_ojo_izq -= np.array([x_min, y_min])
        centro_nariz -= np.array([x_min, y_min])
        
        # Calcular el ángulo de rotación
        dx = centro_ojo_izq[0] - centro_ojo_der[0]
        dy = centro_ojo_izq[1] - centro_ojo_der[1]
        angle = np.degrees(np.arctan2(dy, dx))
        
        # Crear matriz de rotación
        eye_center = ((centro_ojo_der[0] + centro_ojo_izq[0]) // 2,
                      (centro_ojo_der[1] + centro_ojo_izq[1]) // 2)
        rotation_matrix = cv2.getRotationMatrix2D(eye_center, angle, 1.0)
        
        # Rotar la imagen
        image_aligned = cv2.warpAffine(
            image_cropped, rotation_matrix, (original_width, original_height), flags=cv2.INTER_LINEAR)
        
        # Convertir los puntos clave de la nariz
        nose_points = np.array([landmarks[i] for i in nariz], dtype=np.float32)
        nose_points -= [x_min, y_min]

        # Transformar los puntos clave con la misma rotación
        ones = np.ones(shape=(len(nose_points), 1))
        points_ones = np.hstack([nose_points, ones])
        nose_points_rotated = rotation_matrix.dot(points_ones.T).T
        
        # **Nuevo**: Recorte alrededor del punto centrado de la nariz
        centro_x, centro_y = calcular_centro_region(nose_points_rotated, range(len(nariz)))

        # Definir los límites del recorte de 112x112 píxeles
        half_crop_size = 56
        x1 = int(max(0, centro_x - half_crop_size))
        y1 = int(max(0, centro_y - half_crop_size))
        x2 = int(min(original_width, centro_x + half_crop_size))
        y2 = int(min(original_height, centro_y + half_crop_size))
        
        # Recortar la imagen
        image_cropped_nose = image_aligned[y1:y2, x1:x2]
        
        # Redimensionar a 224x224
        image_resized = cv2.resize(image_cropped_nose, (224, 224))

        # Ajustar los puntos faciales de la nariz al nuevo recorte
        nose_points_adjusted = [(p[0] - x1, p[1] - y1) for p in nose_points_rotated]

        # Redimensionar las coordenadas de los puntos a la escala 224x224
        scale_x = 224 / (x2 - x1)
        scale_y = 224 / (y2 - y1)
        nose_points_scaled = [(p[0] * scale_x, p[1] * scale_y) for p in nose_points_adjusted]

        # Aplicar transformaciones de imagen y etiquetas
        if self.transform:
            augmented = self.transform(image=image_resized, keypoints=nose_points_scaled)
            image = augmented['image']
            nose_points = np.array(augmented['keypoints'], dtype=np.float32).flatten()
        
        return image, torch.tensor(nose_points).float(), img_name, original_width, original_height

# Transformaciones de aumento de datos para el conjunto de entrenamiento
train_transforms = A.Compose([
    A.Rotate(limit=30, p=0.5),  # Rotación aleatoria con 50% de probabilidad
    A.ColorJitter(p=0.5),  # Cambio en el balance de color, brillo, contraste y nitidez con 50% de probabilidad
    A.RandomBrightnessContrast(p=0.5),  # Cambio en el brillo y contraste con 50% de probabilidad
    A.GaussianBlur(p=0.1),  # Aplicación de máscaras de desenfoque con 50% de probabilidad
    A.GaussNoise(p=0.1),  # Ruido aleatorio con 50% de probabilidad
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Transformaciones para el conjunto de validación
val_transforms = A.Compose([
    A.Resize(224, 224),  # Redimensionar a 224x224
    A.Normalize(),
    ToTensorV2(),
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# Crear los datasets de entrenamiento y validación
train_dataset = NoseDataset(train_img_dir, train_labels_dir, transform=train_transforms)
val_dataset = NoseDataset(val_img_dir, val_labels_dir, transform=val_transforms)

# Crear DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modelo basado en DenseNet-121
class DenseNet121Nose(nn.Module):
    def __init__(self):
        super(DenseNet121Nose, self).__init__()
        self.backbone = models.densenet121(pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, len(nariz) * 2)
        
    def forward(self, x):
        x = self.backbone(x)
        x = nn.ReLU()(self.fc1(x))
        x = nn.ReLU()(self.fc2(x))
        return self.fc3(x)

# Configuración y entrenamiento
model = DenseNet121Nose().cuda()
optimizer = optim.Adamax(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=75, verbose=True)

def calculate_nme(predictions, ground_truths, num_landmarks):
    """
    Calcula el NME (Normalized Mean Error) utilizando la fórmula específica.
    
    :param predictions: Array de predicciones, de tamaño (N, M, 2), donde
                        N es el número de imágenes, M el número de landmarks, 
                        y 2 son las coordenadas (x, y) de cada punto.
    :param ground_truths: Array de valores reales (ground truths), del mismo tamaño que predictions.
    :param num_landmarks: Número de landmarks en cada imagen (M).
    :return: NME calculado sobre todo el dataset.
    """
    num_images = predictions.shape[0]
    total_error = 0.0

    for i in range(num_images):
        # Verificar que los índices utilizados para calcular la distancia interocular sean correctos
        if len(ground_truths[i]) < 5:
            raise ValueError(f"Expected at least 5 landmarks, but got {len(ground_truths[i])}")

        # Calcula la distancia interocular usando los puntos 0 y 4 (ajustado para nariz)
        iodi = np.linalg.norm(ground_truths[i, 0] - ground_truths[i, 4])
        
        # Suma de errores normalizados por imagen
        for j in range(num_landmarks):
            pred_coord = predictions[i, j]
            gt_coord = ground_truths[i, j]
            error = np.linalg.norm(pred_coord - gt_coord) / iodi
            total_error += error
    
    # Aplicar la fórmula para el NME
    nme = total_error / (num_landmarks * num_images)
    return nme


def train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=None, scheduler=None):
    best_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, nose_points, _, _, _ in train_dataloader:
            images, nose_points = images.cuda(), nose_points.cuda()
            outputs = model(images)
            loss = criterion(outputs, nose_points)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_dataloader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        
        # Validación y cálculo del NME
        if val_dataloader:
            model.eval()
            val_loss = 0.0
            all_labels = []
            all_predictions = []
            with torch.no_grad():
                for val_images, val_nose_points, _, _, _ in val_dataloader:
                    val_images, val_nose_points = val_images.cuda(), val_nose_points.cuda()
                    val_outputs = model(val_images)
                    val_loss += criterion(val_outputs, val_nose_points).item()
                    
                    # Guardar las predicciones y etiquetas reales para el cálculo de NME
                    all_labels.append(val_nose_points.cpu().numpy().reshape(-1, len(nariz), 2))
                    all_predictions.append(val_outputs.cpu().numpy().reshape(-1, len(nariz), 2))

            val_loss /= len(val_dataloader)
            print(f'Validation Loss: {val_loss:.4f}')
            
            # Concatenar todas las predicciones y *ground truths* para el NME
            all_labels = np.concatenate(all_labels, axis=0)
            all_predictions = np.concatenate(all_predictions, axis=0)
            nme = calculate_nme(all_predictions, all_labels, num_landmarks=len(nariz))
            print(f'NME: {nme:.4f}')
            
            # Guardar el mejor modelo
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), 'NoseAreaLandmarks V3.pth')
                print(f'Best model saved at epoch {epoch+1} with val_loss {best_loss:.4f}')
                
            if scheduler:
                scheduler.step(val_loss)

    print("Training complete.")

# Entrenar el modelo
train_model(model, train_dataloader, optimizer, criterion, num_epochs=300, val_dataloader=val_dataloader, scheduler=scheduler)

c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\luisb250\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/300], Loss: 16677.3260
Validation Loss: 16358.2951
NME: 3.4990
Best model saved at epoch 1 with val_loss 16358.2951
Epoch [2/300], Loss: 15202.6198
Validation Loss: 13638.2101
NME: 3.1398
Best model saved at epoch 2 with val_loss 13638.2101
Epoch [3/300], Loss: 11017.5939
Validation Loss: 8050.1736
NME: 2.2835
Best model saved at epoch 3 with val_loss 8050.1736
Epoch [4/300], Loss: 5194.3109
Validation Loss: 2993.6185
NME: 1.1581
Best model saved at epoch 4 with val_loss 2993.6185
Epoch [5/300], Loss: 2073.6053
Validation Loss: 1498.7568
NME: 0.6843
Best model saved at epoch 5 with val_loss 1498.7568
Epoch [6/300], Loss: 1288.4567
Validation Loss: 1072.7748
NME: 0.5406
Best model saved at epoch 6 with val_loss 1072.7748
Epoch [7/300], Loss: 1025.6153
Validation Loss: 926.0035
NME: 0.5155
Best model saved at epoch 7 with val_loss 926.0035
Epoch [8/300], Loss: 950.1219
Validation Loss: 876.0608
NME: 0.5259
Best model saved at epoch 8 with val_loss 876.0608
Epoch [9/300], Loss: 9